In [1]:
import pandas as pd
import numpy as np
train_data = pd.read_csv("../input/fmnlpca2dataset/train.csv")
test_data = pd.read_csv("../input/fmnlpca2dataset/test.csv")
train_data = train_data[['app_id', 'description_fa', 'label']]
test_data = test_data[['app_id', 'description_fa', 'label']]
train_data = train_data.dropna()

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (19,21,23,25,27,29,31,33,35,37,39,41,43,45,47,49,51,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [2]:
train_data = train_data[train_data['label'].astype(str).str.isnumeric() == True]
train_data

,app_id,description_fa,label
0,7278,<p> <b>آموزش نحوه راه اندازی یک مرغ داری شامل ...,8
1,24271,<p>دلتون می خواد با شخصیت های معروف دنیا سلفی ...,7
2,37332,"<p>بازی ""شکارچی"" یک بازی سرگرم کننده و مبتنی ب...",6
3,12139,<p>به راحتی اسم خود و عزیزانتان را به صورت کام...,7
4,25387,<p>در این بازی با شخصیت گارفیلد به ماجراجویی خ...,4
...,...,...,...
30309,17517,<p>تاحالا ماهی گیری کردین؟! واقعا؟!! کجا؟َ! تو...,6
30310,25380,<p>«<b>همگو</b>»؛ یک شبکه‌ی اجتماعی فارسی زبان...,9
30311,24025,<p>بازی مرد عنکبوتی یکی از بازی های سبک هیجان...,4
30312,25921,<p>توضیح</p>\n<p> </p>\n<p> </p>\n<p>یک برنامه...,9


In [3]:
labels_train = [int(label) for label in train_data['label']]

**wordpiece_tokenizer**: tokenized into words.

**subwordBased_tokenizer**: Subword tokenization algorithms rely on the principle that frequently used words should not be split into smaller subwords, but rare words should be decomposed into meaningful subwords. For instance "annoyingly" might be considered a rare word and could be decomposed into "annoying" and "ly". 

**XLNET_tokenizer**: The XLNetTokenizer uses SentencePiece for example, which is also why in the example earlier the "▁" character was included in the vocabulary. Decoding with SentencePiece is very easy since all tokens can just be concatenated and "▁" is replaced by a space.

In [4]:
from transformers import AutoConfig, AutoTokenizer, AutoModel, TFAutoModel, XLNetTokenizer
wordpiece_tokenizer = AutoTokenizer.from_pretrained("HooshvareLab/bert-fa-zwnj-base")
XLNET_tokenizer = XLNetTokenizer.from_pretrained("xlnet-base-cased")

Downloading:   0%|          | 0.00/292 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/565 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/416k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.06M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/134 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/779k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.32M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/760 [00:00<?, ?B/s]

In [5]:
!pip install hazm

     |████████████████████████████████| 316 kB 4.3 MB/s            
     |████████████████████████████████| 233 kB 50.9 MB/s            
  Preparing metadata (setup.py) ... done
     |████████████████████████████████| 1.4 MB 60.9 MB/s            
  Preparing metadata (setup.py) ... done
  Created wheel for nltk: filename=nltk-3.3-py3-none-any.whl size=1394485 sha256=1670af4c8b04faaaabc2e1ee663f6ae58e18e5f069b3093b89fdef8d2c964929
  Stored in directory: /root/.cache/pip/wheels/9b/fd/0c/d92302c876e5de87ebd7fc0979d82edb93e2d8d768bf71fac4
  Created wheel for libwapiti: filename=libwapiti-0.2.1-cp37-cp37m-linux_x86_64.whl size=192823 sha256=8a6b3606bbffb100310536a3606e3abfab22fb9054b9aeba176539697599040f
  Stored in directory: /root/.cache/pip/wheels/ab/b2/5b/0fe4b8f5c0e65341e8ea7bb3f4a6ebabfe8b1ac31322392dbf
Successfully built nltk libwapiti
  Attempting uninstall: nltk
    Found existing installation: nltk 3.2.4
    Uninstalling nltk-3.2.4:
      Successfully uninstalled nltk-3.2.4
ERROR:

<h3> Here, we create 2 tokenized datasets: first is uses word piece tokenizer, and the second one uses XLNET tokenizer </h3>

In [6]:
import re
from hazm import *
descreptions = train_data['description_fa']
all_embedded_data_wordpiece = []
all_embedded_data_XLNET = []
s1 = []
s2 = []
for desc in descreptions:

    desc = re.sub(r'[@|#][A-Za-z-0-9_]*|(https:[A-Za-z-0-9_:/.]*)|<p>|</p>|<b>|</br>|</b>|<br>', '', desc)

    tokenized_to_word_wordpiece = wordpiece_tokenizer.encode(desc, add_special_tokens=True,)
    all_embedded_data_wordpiece.append(tokenized_to_word_wordpiece)
    s1.append(len(tokenized_to_word_wordpiece))
    
    tokenized_to_word_XLNET = XLNET_tokenizer.encode(desc, add_special_tokens=True,)
    all_embedded_data_XLNET.append(tokenized_to_word_XLNET)
    s2.append(len(tokenized_to_word_XLNET))

Token indices sequence length is longer than the specified maximum sequence length for this model (655 > 512). Running this sequence through the model will result in indexing errors


# Now, let's see what the average text sizes are in both tokenizers

In [7]:
avg_text_size = sum(s1)/len(s1)
print("average text size with word piece tokenizer: ", avg_text_size)
avg_text_size = sum(s2)/len(s2)
print("average text size with XLNET tokenizer: ", avg_text_size)

average text size with word piece tokenizer:  178.12353233008102
average text size with XLNET tokenizer:  399.2922771622292


As we can see, there are much less tokenized words in the word piece tokenizer.Moreover, the average text len is less than 512 which is a good sign to use the bert model with nothing to be worried about

# Adding paddings and creating attention masks
* after running the cell bellow, all texts have the same size of 512

* pay attention that we do this for both of our tokenizers in order to compare their results 

In [8]:
def add_padding_attentionMask(all_embedded_data):
    all_attention_mask = []
    for i in range(len(all_embedded_data)):
        attention_mask_text = []
        if len(all_embedded_data[i]) < 512:
            all_embedded_data[i] = all_embedded_data[i] + [0] * (512-len(all_embedded_data[i]))
        else:
            all_embedded_data[i] = all_embedded_data[i][:511] + [102]

        for num in range(len(all_embedded_data[i])):
            if all_embedded_data[i][num] > 0:
                attention_mask_text.append(1)
            else:
                attention_mask_text.append(0)  
        all_attention_mask.append(attention_mask_text) 
    
    return all_embedded_data, all_attention_mask
all_embedded_data_wordpiece, attentionMask_wordpiece = add_padding_attentionMask(all_embedded_data_wordpiece) 
all_embedded_data_XLNET, attentionMask_XLNET = add_padding_attentionMask(all_embedded_data_XLNET) 

# preparing input for model

In [9]:
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset,DataLoader, RandomSampler, SequentialSampler
from transformers import BertForSequenceClassification, AdamW, BertConfig
import torch

def preparing_input_for_model(all_embedded_data, all_attention_mask, batch_size):
    #spliting dataset
    train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(all_embedded_data, labels_train, random_state = 2018, test_size=0.1)
    train_mask, validation_mask, _, _ = train_test_split(all_attention_mask, labels_train, random_state = 2018, test_size=0.1)
    
    #changing input format to tensor
    train_inputs = torch.tensor(train_inputs)
    validation_inputs = torch.tensor(validation_inputs)
    train_labels = torch.tensor(train_labels)
    validation_labels = torch.tensor(validation_labels)
    train_mask = torch.tensor(train_mask)
    validation_mask = torch.tensor(validation_mask)

    # Create the DataLoaders for our training and validation sets.
    # We'll take training samples in random order.
    train_dataset = TensorDataset(train_inputs, train_mask, train_labels)
    train_dataloader = DataLoader(
                train_dataset,  # The training samples.
                sampler = RandomSampler(train_dataset), # Select batches randomly
                batch_size = batch_size # Trains with this batch size.
            )

    # For validation the order doesn't matter, so we'll just read them sequentially.
    validation_dataset = TensorDataset(validation_inputs, validation_mask, validation_labels)
    validation_dataloader = DataLoader(
                validation_dataset, # The validation samples.
                sampler = SequentialSampler(validation_dataset), # Pull out batches sequentially.
                batch_size = batch_size # Evaluate with this batch size.
            )
    
    return train_dataloader, validation_dataloader
    

<h4>Two functions which are called in our training process</h4>

In [10]:
import numpy as np
import time
import datetime

# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

<h4>This is the function which is going to be called several times to train different models</h4>

<h4>Model is created and trained in several batches and epochs</h4>

In [11]:
from transformers import get_linear_schedule_with_warmup
import random
import torch
import pandas as pd

def train_the_model(all_embedded_data=all_embedded_data_wordpiece, all_attention_mask=attentionMask_wordpiece, batch_size=16, model_name="HooshvareLab/bert-fa-zwnj-base", learning_rate=2e-5, epochs=1):
    train_dataloader, validation_dataloader = preparing_input_for_model(all_embedded_data, all_attention_mask, batch_size)
    model = BertForSequenceClassification.from_pretrained(
                model_name, # Use the 12-layer BERT model, with an uncased vocab.
                num_labels = 10, # The number of output labels--2 for binary classification.  
                output_attentions = False, # Whether the model returns attentions weights.
                output_hidden_states = False, # Whether the model returns all hidden-states.
            )
    
    model.cuda()
    
    optimizer = AdamW(model.parameters(),
                  lr = learning_rate, # args.learning_rate - default is 5e-5, our notebook had 2e-5
                  eps = 1e-8 # args.adam_epsilon  - default is 1e-8.
                )
    
    # Create the learning rate scheduler.
    total_steps = len(train_dataloader) * epochs
    scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps = 0,num_training_steps = total_steps)
    
    
    seed_val = 42
    random.seed(seed_val)
    np.random.seed(seed_val)
    torch.manual_seed(seed_val)
    torch.cuda.manual_seed_all(seed_val)

    training_stats = []
    total_t0 = time.time()
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    for epoch_i in range(0, epochs):

        # ========================================
        #               Training
        # ========================================

        # Perform one full pass over the training set.

        print("")
        print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
        print('Training...')

        # Measure how long the training epoch takes.
        t0 = time.time()

        # Reset the total loss for this epoch.
        total_train_loss = 0

        # Put the model into training mode.
        model.train()

        # For each batch of training data...
        for step, batch in enumerate(train_dataloader):

            # Progress update every 40 batches.
            if step % 40 == 0 and not step == 0:
                # Calculate elapsed time in minutes.
                elapsed = format_time(time.time() - t0)

                # Report progress.
                print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

            
            # Unpack this training batch from our dataloader. 
            # As we unpack the batch, we'll also copy each tensor to the GPU using the `to` method.
            b_input_ids = batch[0].to(device)
            b_input_mask = batch[1].to(device)
            b_labels = batch[2].to(device)

            # Always clear any previously calculated gradients before performing a backward pass. 
            model.zero_grad()        

            # Perform a forward pass (evaluate the model on this training batch).
            # it returns the loss (because we provided labels) and 
            # the "logits"--the model outputs prior to activation.
            outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)

            # Accumulate the training loss over all of the batches so that we can
            # calculate the average loss at the end. `loss` is a Tensor containing a
            # single value; the `.item()` function just returns the Python value 
            # from the tensor.
            loss = outputs[0]
            total_train_loss += loss.item()

            # Perform a backward pass to calculate the gradients.
            loss.backward()

            # Clip the norm of the gradients to 1.0.
            # This is to help prevent the "exploding gradients" problem.
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

            # Update parameters and take a step using the computed gradient.
            # The optimizer dictates the "update rule"--how the parameters are
            # modified based on their gradients, the learning rate, etc.
            optimizer.step()

            # Update the learning rate.
            scheduler.step()

        # Calculate the average loss over all of the batches.
        avg_train_loss = total_train_loss / len(train_dataloader)            

        # Measure how long this epoch took.
        training_time = format_time(time.time() - t0)

        print("")
        print("  Average training loss: {0:.2f}".format(avg_train_loss))
        print("  Training epcoh took: {:}".format(training_time))

        # ========================================
        #               Validation
        # ========================================
        # After the completion of each training epoch, measure our performance on
        # our validation set.

        print("")
        print("Running Validation...")

        t0 = time.time()

        # Put the model in evaluation mode--the dropout layers behave differentl during evaluation.
        model.eval()

        # Tracking variables 
        total_eval_accuracy = 0
        total_eval_loss = 0
        nb_eval_steps = 0

        # Evaluate data for one epoch
        for batch in validation_dataloader:

            # Unpack this training batch from our dataloader. 
            b_input_ids = batch[0].to(device)
            b_input_mask = batch[1].to(device)
            b_labels = batch[2].to(device)

            # Tell pytorch not to bother with constructing the compute graph during
            # the forward pass, since this is only needed for backprop (training).
            with torch.no_grad():        

                # Forward pass, calculate logit predictions.
                # token_type_ids is the same as the "segment ids", which 
                # differentiates sentence 1 and 2 in 2-sentence tasks.
                # Get the "logits" output by the model. The "logits" are the output
                # values prior to applying an activation function like the softmax.
                outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)

            # Accumulate the validation loss.
            logits = outputs[1]

            # Move logits and labels to CPU
            logits = logits.detach().cpu().numpy()
            label_ids = b_labels.to('cpu').numpy()

            # Calculate the accuracy for this batch of test sentences, and accumulate it over all batches.
            total_eval_accuracy += flat_accuracy(logits, label_ids)

            nb_eval_steps += 1

        # Report the final accuracy for this validation run.
        avg_val_accuracy = total_eval_accuracy / nb_eval_steps
        print("  Accuracy: {0:.2f}".format(avg_val_accuracy))

        # Calculate the average loss over all of the batches.
        avg_val_loss = total_eval_loss / len(validation_dataloader)

        # Measure how long the validation run took.
        validation_time = format_time(time.time() - t0)

        print("  Validation Loss: {0:.2f}".format(avg_val_loss))
        print("  Validation took: {:}".format(validation_time))

        # Record all statistics from this epoch.
        training_stats.append(
            {
                'epoch': epoch_i + 1,
                'Training Loss': avg_train_loss,
                'Valid. Loss': avg_val_loss,
                'Valid. Accur.': avg_val_accuracy,
                'Training Time': training_time,
                'Validation Time': validation_time
            }
        )

    print("")
    print("Training complete!")

    print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))

    
    pd.set_option('precision', 2)
    df_stats = pd.DataFrame(data=training_stats)
    df_stats = df_stats.set_index('epoch')
    return model, df_stats

# Let's train our model with different hyper parameters

# 1) base version

data: word piece

batch size: 16

BERT model: parseBert

learning rate: 2e-5

epoch: 1

In [ ]:
train_the_model(all_embedded_data_wordpiece, attentionMask_wordpiece, 16, "HooshvareLab/bert-fa-zwnj-base", 2e-5, 1)

# 2) Now let's change the tokenizer from word piece to XLNET

data: XLNET

batch size: 16

BERT model: parseBert

learning rate: 2e-5

epoch: 1

In [ ]:
train_the_model(all_embedded_data_XLNET, attentionMask_XLNET, 16, "HooshvareLab/bert-fa-zwnj-base", 2e-5, 1)

<h4> As we can see, the loss of word piece tokenizer is less than the XLNET tokenizer; therefore, we choose word piece tokenizer </h4>

# 3) Now it is time to change the bert model from parseBert into multilingualBert

data: **word piece**

batch size: 16

BERT model: multilingual

learning rate: 2e-5

epoch: 1

In [ ]:
train_the_model(all_embedded_data_wordpiece, attentionMask_wordpiece, 16, "bert-base-multilingual-cased", 2e-5, 1)

<h4> As we can see, the loss of parseBert model is less than the multilingualBert model; therefore, we choose the parseBert model </h4>

# 4) Now it is time to change the learning rate from 2e-5 to 2e-4

data: **word piece**

batch size: 16

BERT model: **parseBert**

learning rate: 2e-4

epoch: 1

In [ ]:
train_the_model(all_embedded_data_wordpiece, attentionMask_wordpiece, 16, "HooshvareLab/bert-fa-zwnj-base", 2e-4, 1)

<h4> As we can see, loss increases </h4>

# 5) Now it is time to change the learning rate from 2e-5 to 2e-6

data: **word piece**

batch size: 16

BERT model: **parseBert**

learning rate: 2e-6

epoch: 1

In [ ]:
train_the_model(all_embedded_data_wordpiece, attentionMask_wordpiece, 16, "HooshvareLab/bert-fa-zwnj-base", 2e-6, 1)

<h4> As we can see, loss increases </h4>

# 6) Now it is time to change the learning rate from 2e-5 to 5e-5

data: **word piece**

batch size: 16

BERT model: **parseBert**

learning rate: 5e-5

epoch: 1

In [ ]:
train_the_model(all_embedded_data_wordpiece, attentionMask_wordpiece, 16, "HooshvareLab/bert-fa-zwnj-base", 5e-5, 1)

<h4> As we can see, loss decreases </h4>

# 7) Now it is time to change the batch size to 8

data: **word piece**

batch size: 8

BERT model: **parseBert**

learning rate: **5e-5**

epoch: 1

In [ ]:
train_the_model(all_embedded_data_wordpiece, attentionMask_wordpiece, 8, "HooshvareLab/bert-fa-zwnj-base", 5e-5, 1)

<h4> As we can see, loss increases </h4>

# 8) Now it is time to change the epoch to 5

data: **word piece**

batch size: **16**

BERT model: **parseBert**

learning rate: **5e-5**

epoch: **5**

In [12]:
final_model, df_stats = train_the_model(all_embedded_data_wordpiece, attentionMask_wordpiece, 16, "HooshvareLab/bert-fa-zwnj-base", 5e-5, 6)

Downloading:   0%|          | 0.00/452M [00:00<?, ?B/s]

Some weights of the model checkpoint at HooshvareLab/bert-fa-zwnj-base were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at HooshvareLab/b


======== Epoch 1 / 6 ========
Training...
  Batch    40  of  1,701.    Elapsed: 0:00:35.
  Batch    80  of  1,701.    Elapsed: 0:01:09.
  Batch   120  of  1,701.    Elapsed: 0:01:43.
  Batch   160  of  1,701.    Elapsed: 0:02:17.
  Batch   200  of  1,701.    Elapsed: 0:02:51.
  Batch   240  of  1,701.    Elapsed: 0:03:26.
  Batch   280  of  1,701.    Elapsed: 0:04:00.
  Batch   320  of  1,701.    Elapsed: 0:04:34.
  Batch   360  of  1,701.    Elapsed: 0:05:08.
  Batch   400  of  1,701.    Elapsed: 0:05:42.
  Batch   440  of  1,701.    Elapsed: 0:06:16.
  Batch   480  of  1,701.    Elapsed: 0:06:50.
  Batch   520  of  1,701.    Elapsed: 0:07:24.
  Batch   560  of  1,701.    Elapsed: 0:07:59.
  Batch   600  of  1,701.    Elapsed: 0:08:33.
  Batch   640  of  1,701.    Elapsed: 0:09:07.
  Batch   680  of  1,701.    Elapsed: 0:09:41.
  Batch   720  of  1,701.    Elapsed: 0:10:15.
  Batch   760  of  1,701.    Elapsed: 0:10:49.
  Batch   800  of  1,701.    Elapsed: 0:11:23.
  Batch   840  of

In [13]:
df_stats

,Training Loss,Valid. Loss,Valid. Accur.,Training Time,Validation Time
epoch,,,,,
1,0.90,0.0,0.74,0:24:12,0:00:54
2,0.61,0.0,0.75,0:24:13,0:00:54
3,0.41,0.0,0.75,0:24:14,0:00:54
4,0.24,0.0,0.75,0:24:14,0:00:54
5,0.14,0.0,0.74,0:24:14,0:00:54
6,0.07,0.0,0.74,0:24:13,0:00:54


In [14]:
import matplotlib.pyplot as plt
% matplotlib inline

import seaborn as sns

# Use plot styling from seaborn.
sns.set(style='darkgrid')

# Increase the plot size and font size.
sns.set(font_scale=1.5)
plt.rcParams["figure.figsize"] = (12,6)

# Plot the learning curve.
plt.plot(df_stats['Training Loss'], 'b-o', label="Training")
plt.plot(df_stats['Valid. Loss'], 'g-o', label="Validation")

# Label the plot.
plt.title("Training & Validation Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()
plt.xticks([1, 2, 3, 4])

plt.show()

UsageError: Line magic function `%` not found.


In [ ]:
import pandas as pd

# Load the dataset into a pandas dataframe.
df = test_data
# Report the number of sentences.
print('Number of test sentences: {:,}\n'.format(df.shape[0]))

# Create sentence and label lists
descriptions = df.description_fa.values
test_data = test_data[test_data['label'].astype(str).str.isnumeric() == True]
labels = test_data['label']
# Tokenize all of the sentences and map the tokens to thier word IDs.
labels_test = [int(label) for label in labels]

input_ids = []
# attention_masks = []
# # For every sentence...
for desc in descriptions:
    desc = re.sub(r'[@|#][A-Za-z-0-9_]*|(https:[A-Za-z-0-9_:/.]*)|<p>|</p>|<b>|</br>|</b>|<br>', '', sent)
    encoded_desc = tokenizer.encode(desc, add_special_tokens=True,)
    input_ids.append(encoded_desc)

    
all_attention_mask = []
for i in range(len(input_ids)):
    attention_mask_text = []
    if len(input_ids[i]) < 512:
        input_ids[i] = input_ids[i] + [0] * (512-len(input_ids[i]))
    else:
        input_ids[i] = input_ids[i][:511] + [102]
    
    for num in range(len(input_ids[i])):
        if input_ids[i][num] > 0:
            attention_mask_text.append(1)
        else:
            attention_mask_text.append(0)  
    all_attention_mask.append(attention_mask_text) 
    
prediction_inputs = torch.tensor(input_ids)
prediction_masks = torch.tensor(all_attention_mask)
prediction_labels = torch.tensor(labels_test)
print(len(prediction_inputs), len(prediction_masks), len(prediction_labels))

batch_size = 16


test_dataset = TensorDataset(prediction_inputs, prediction_masks, prediction_labels)
test_dataloader = DataLoader(
            test_dataset,  # The training samples.
            sampler = RandomSampler(test_dataset), # Select batches randomly
            batch_size = batch_size # Trains with this batch size.
        )


In [ ]:
# Prediction on test set
print('Predicting labels for {:,} test sentences...'.format(len(input_ids)))

# Put model in evaluation mode
model.eval()

# Tracking variables 
predictions , true_labels = [], []

# Predict 
for batch in test_dataloader:
  # Add batch to GPU
    batch = tuple(t.to(device) for t in batch)
  
    b_input_ids, b_input_mask, b_labels = batch
    
  
  # Telling the model not to compute or store gradients, saving memory and 
  # speeding up prediction
    with torch.no_grad():
      # Forward pass, calculate logit predictions
        outputs = model(b_input_ids, token_type_ids=None, 
                      attention_mask=b_input_mask)
    
    logits = outputs[0]
    print(len(outputs))
  # Move logits and labels to CPU
    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()

    # Store predictions and true labels
    predictions.append(logits)
    true_labels.append(label_ids)

print('    DONE.')

In [ ]:
from sklearn.metrics import matthews_corrcoef

matthews_set = []

# Evaluate each test batch using Matthew's correlation coefficient
print('Calculating Matthews Corr. Coef. for each batch...')

# For each input batch...
for i in range(len(true_labels)):
  
    # The predictions for this batch are a 2-column ndarray (one column for "0" 
    # and one column for "1"). Pick the label with the highest value and turn this
    # in to a list of 0s and 1s.
    

    pred_labels_i = np.argmax(predictions[i], axis=1).flatten()

    # Calculate and store the coef for this batch.  
    matthews = matthews_corrcoef(true_labels[i], pred_labels_i)                
    matthews_set.append(matthews)

In [ ]:
matthews_set

In [ ]:
# Combine the results across all batches. 
flat_predictions = np.concatenate(predictions, axis=0)

# For each sample, pick the label (0 or 1) with the higher score.
flat_predictions = np.argmax(flat_predictions, axis=1).flatten()

# Combine the correct labels for each batch into a single list.
flat_true_labels = np.concatenate(true_labels, axis=0)

# Calculate the MCC
mcc = matthews_corrcoef(flat_true_labels, flat_predictions)

print('Total MCC: %.3f' % mcc)